<a href="https://colab.research.google.com/github/TrieuPhi/Huggingface-Captioning-Data/blob/main/LLM_Ngok_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q transformers==4.44.2 bitsandbytes
!pip install -U -q huggingface_hub
!pip install -q flask flask-cors pyngrok flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [ ]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer
import requests

# Thư viện xử lý ảnh đầu vào

In [ ]:
# Thư viện xử lý ảnh đầu vào (từ source gốc HF)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(requests.get(image_file, stream=True).raw).convert('RGB')#Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values


# Load model và test model trên Colab

In [ ]:
model_name = "5CD-AI/Vintern-1B-v2"
model = AutoModel.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
).eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
generation_config = dict(max_new_tokens= 512, do_sample=False, num_beams = 3, repetition_penalty=3.5)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/5.33k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/5CD-AI/Vintern-1B-v2:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/5CD-AI/Vintern-1B-v2:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.38M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

In [ ]:
test_image = 'https://media-cdn-v2.laodong.vn/Storage/NewsPortal/2022/12/7/1124909/Karaoke-2.jpg'

pixel_values = load_image(test_image, max_num=6).to(torch.bfloat16).cuda()

prompt = '''<image>\nNhận diện hoá đơn trong ảnh. Chỉ trả về phần liệt kê các mặt hàng hàng dưới dạng JSON:
[
  {
    "Tên món": "Tên món",
    "Số lượng": "Số lượng",
    "Đơn giá": "Đơn giá",
    "Thành tiền": "Thành tiền"
  },
]
'''
response = model.chat(tokenizer, pixel_values, prompt, generation_config)

del pixel_values
response

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


'[\n  {\n    "Tên món": "Giờ VIP222",\n    "Số lượng": "1h54\'",\n    "Đơn giá": "500 000",\n    "Thành tiền": "950 000"\n  },\n  {\n    "Tên món": "Suối",\n    "Số lượng": "3",\n    "Đơn giá": "12 000",\n    "Thành tiền": "36 000"\n  },\n  {\n    "Tên món": "Hoa quả thập cẩm",\n    "Số lượng": "1",\n    "Đơn giá": "140 000",\n    "Thành tiền": "140 000"\n  },\n  {\n    "Tên món": "Hoa quả Bưởi",\n    "Số lượng": "2",\n    "Đơn giá": "220 000",\n    "Thành tiền": "440 000"\n  },\n  {\n    "Tên món": "Hoa Quả Roi",\n    "Số lượng": "1",\n    "Đơn giá": "100 000",\n    "Thành tiền": "100 000"\n  },\n  {\n    "Tên món": "Ken ngoại",\n    "Số lượng": "14",\n    "Đơn giá": "60 000",\n    "Thành tiền": "840 000"\n  }\n]'

In [ ]:
test_image = 'https://uploads.nguoidothi.net.vn/content/a0a1cfac-64c2-47eb-a750-097a09008ccd.jpg'

pixel_values = load_image(test_image, max_num=6).to(torch.bfloat16).cuda()

prompt = '''<image>\nMô tả hình ảnh một cách chi tiết, trả về dạng văn bản.'''
response = model.chat(tokenizer, pixel_values, prompt, generation_config)

del pixel_values
response

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


'Bức ảnh chụp một con phố ở Việt Nam bị lũ lụt. Phía trước là một người đàn ông mặc áo mưa màu đỏ đang lái xe máy, phía sau là một nhóm người đang ngồi trên xe máy. Phía bên phải của bức ảnh là một cửa hàng có biển hiệu "Kim Oanh" với dòng chữ tiếng Anh "Kim Oanh". Phía bên trái của bức ảnh là một cửa hàng có biển hiệu "Công ty Cổ phần Thương mại Phát Hỗ Trợ" với dòng chữ tiếng Anh "Công ty Cổ phần Thương mại Phát Hỗ Trợ".  Bức ảnh được chụp từ góc nhìn nghiêng, cho thấy toàn cảnh con phố bị lũ lụt.'

# Triển khai Flask và Expose ra API qua Ngrok

In [ ]:
# Setup Ngrok Token
from google.colab import userdata
from flask import Flask, jsonify, request
from flask_cors import CORS
from pyngrok import ngrok

authtoken = userdata.get("ngrok_token")
ngrok.set_auth_token(authtoken)

In [ ]:
# Viết code Flask để expose ra API

# Initialize Flask app
app = Flask(__name__)
CORS(app)


prompt = '''<image>\nMô tả hình ảnh một cách chi tiết, trả về dạng văn bản.'''


@app.route('/imc', methods=['POST'])
def index():
    data = request.json
    image_url = data.get('image_url', None)

    response_message = ocr_by_llm(image_url, prompt)

    return jsonify({
        "response_message": response_message
    })

def ocr_by_llm(image_url, prompt):
    # image = Image.open(requests.get(image_url, stream=True).raw)

    pixel_values = load_image(image_url, max_num=6).to(torch.bfloat16).cuda()

    response_message = model.chat(tokenizer, pixel_values, prompt, generation_config)

    del pixel_values

    print(response_message)
    return response_message

if __name__ == '__main__':

    ngrok_url = ngrok.connect(5555)
    print(ngrok_url)

    app.run(port=5555)

NgrokTunnel: "https://e555-34-125-45-105.ngrok-free.app" -> "http://localhost:5555"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5555
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 11:02:44] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 11:02:44] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 11:02:59] "POST / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 11:06:18] "POST / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 11:06:46] "POST / HTTP/1.1" 404 -
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 11:07:30] "POST /ocr HTTP/1.1" 200 -


Bức ảnh chụp một con phố ở Việt Nam bị lũ lụt. Phía trước là một người đàn ông mặc áo mưa màu đỏ đang lái xe máy, phía sau là một nhóm người đang ngồi trên xe máy. Phía bên phải của bức ảnh là một cửa hàng có biển hiệu "Kim Oanh" với dòng chữ tiếng Anh "Kim Oanh". Phía bên trái của bức ảnh là một cửa hàng có biển hiệu "Công ty Cổ phần Thương mại Phát Hỗ Trợ" với dòng chữ tiếng Anh "Công ty Cổ phần Thương mại Phát Hỗ Trợ".  Bức ảnh được chụp từ góc nhìn nghiêng, cho thấy toàn cảnh con phố bị lũ lụt.


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 11:08:37] "POST /ocr HTTP/1.1" 200 -


Bức ảnh chụp một con phố ở Việt Nam bị lũ lụt. Phía trước là một người đàn ông mặc áo mưa màu đỏ đang lái xe máy, phía sau là một nhóm người đang ngồi trên xe máy. Phía bên phải của bức ảnh là một cửa hàng có biển hiệu "Kim Oanh" với dòng chữ tiếng Anh "Kim Oanh". Phía bên trái của bức ảnh là một cửa hàng có biển hiệu "Công ty Cổ phần Thương mại Phát Hỗ Trợ" với dòng chữ tiếng Anh "Công ty Cổ phần Thương mại Phát Hỗ Trợ".  Bức ảnh được chụp từ góc nhìn nghiêng, cho thấy toàn cảnh con phố bị lũ lụt.


...

Bức ảnh chụp từ trên cao xuống một con đường đông đúc với nhiều phương tiện giao thông, chủ yếu là ô tô và xe máy. Đường phố được bao quanh bởi những cây xanh và các tòa nhà cao tầng. Phía bên phải của bức ảnh, có thể nhìn thấy một cây cầu vượt đang xây dựng.  Bức ảnh cho thấy sự nhộn nhịp của cuộc sống ở Việt Nam, đặc biệt là tại thành phố Hồ Chí Minh.


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 12:29:31] "POST /ocr HTTP/1.1" 200 -


Bức ảnh được chia thành 5 phần, mỗi phần là một hình ảnh khác nhau. 

Phần trên bên trái: Hình ảnh một con phố với nhiều cửa hàng và xe cộ đang lưu thông. Trên tường có dòng chữ "106.5 MHz VOT GIỜ CAO ĐIỂM GIAO THÔNG" và "LIVE NOW ON AIR".

Phần trên bên phải: Hình ảnh một người phụ nữ đang ngồi nghe truyền hình trên máy tính xách tay. Phía sau cô ấy là một bảng hiệu quảng cáo với dòng chữ "HÀNG SƠN HÀ NỘI - TRUYỀN THÔNG".

Phần dưới bên trái: Hình ảnh một con đường với nhiều xe cộ đang lưu thông. Trên vỉa hè có đèn tín hiệu giao thông màu xanh lá cây.

Phần dưới bên phải: Hình ảnh một nhóm người đang làm việc tại một văn phòng. Có thể thấy bàn làm việc của họ với các thiết bị điện tử như máy tính, bàn phím và chuột.


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 12:29:47] "POST /ocr HTTP/1.1" 200 -


Bức ảnh chụp từ trên cao, góc nghiêng, cho thấy một ngã tư đường đông đúc với nhiều phương tiện giao thông đang lưu thông. Phía bên trái của bức ảnh là một tòa nhà màu vàng với dòng chữ "thegioi" được in nổi bật. Ở giữa bức ảnh là một con đường với nhiều xe máy và ô tô đang lưu thông. Phía bên phải của bức ảnh là một hàng cây xanh rợp bóng mát. Trên vỉa hè có một biển báo cấm đậu xe.
